In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
# Use Pandas to load dataset from csv file
import pandas as pd

# Make sure we don't get any GPU errors
#physical_devices = tf.config.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# Check the shape of the training and test data
print("Train Images Shape:", train_images.shape)
print("Test Images Shape:", test_images.shape)
print("Train Labels Shape:", train_labels.shape)
print("Test Labels Shape:", test_labels.shape)

Train Images Shape: (60000, 28, 28)
Test Images Shape: (10000, 28, 28)
Train Labels Shape: (60000,)
Test Labels Shape: (10000,)


In [4]:
train_images = train_images.reshape(-1,28,28,1).astype("float32") / 255.0
test_images = test_images.reshape(-1,28,28,1).astype("float32") / 255.0

In [5]:
train_images.shape

(60000, 28, 28, 1)

In [6]:
class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding="same")
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x

In [7]:
class RESBLOCK (layers.Layer):
    def __init__(self, channels):
        super(RESBLOCK, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0], 3)
        self.cnn2 = CNNBlock(channels[1], 3)
        self.cnn3 = CNNBlock(channels[2], 3)
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding="same")

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training,)
        x = self.pooling(x)
        return x

In [8]:
class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = RESBLOCK([32, 32, 64])
        self.block2 = RESBLOCK([128, 128, 256])
        self.block3 = RESBLOCK([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.classifier(x)
        return x

    def model(self):
        x = keras.Input(shape=(28, 28, 1))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [9]:
model = ResNet_Like().model()
base_input = model.layers[0].input
base_output = model.layers[2].output
output = layers.Dense(10)(layers.Flatten()(base_output))
model = keras.Model(base_input, output)

In [10]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

In [11]:
model.fit (train_images, train_labels, batch_size=64, epochs=5, verbose =2 )

Epoch 1/5
938/938 - 27s - loss: 0.1049 - accuracy: 0.9673 - 27s/epoch - 29ms/step
Epoch 2/5
938/938 - 15s - loss: 0.0394 - accuracy: 0.9876 - 15s/epoch - 16ms/step
Epoch 3/5
938/938 - 16s - loss: 0.0292 - accuracy: 0.9906 - 16s/epoch - 17ms/step
Epoch 4/5
938/938 - 16s - loss: 0.0230 - accuracy: 0.9926 - 16s/epoch - 17ms/step
Epoch 5/5
938/938 - 16s - loss: 0.0188 - accuracy: 0.9942 - 16s/epoch - 17ms/step


In [12]:
model.evaluate(test_images, test_labels, batch_size=64, verbose =2 )

157/157 - 2s - loss: 0.0485 - accuracy: 0.9870 - 2s/epoch - 10ms/step


[0.04851662367582321, 0.9869999885559082]